In [1]:
COCO_TRAIN_LABELS = { 'person': [], 'bicycle': [], 'car': [], 'motorcycle': [], 'airplane': [], 'bus': [], 'train': [], 'truck': [], 
               'boat': [], 'traffic light': [], 'fire hydrant': [], 'stop sign': [], 'parking meter': [], 'bench': [], 'bird': [], 
               'cat': [], 'dog': [], 'horse': [], 'sheep': [], 'cow': [], 'elephant': [], 'bear': [], 'zebra': [], 'giraffe': [], 
               'backpack': [], 'umbrella': [], 'handbag': [], 'tie': [], 'suitcase': [], 'frisbee': [], 'skis': [], 'snowboard': [], 
               'sports ball': [], 'kite': [], 'baseball bat': [], 'baseball glove': [], 'skateboard': [], 'surfboard': [], 'tennis racket': [], 
               'bottle': [], 'wine glass': [], 'cup': [], 'fork': [], 'knife': [], 'spoon': [], 'bowl': [], 'banana': [], 'apple': [], 
               'sandwich': [], 'orange': [], 'broccoli': [], 'carrot': [], 'hot dog': [], 'pizza': [], 'donut': [], 'cake': [], 'chair': [], 
               'couch': [], 'potted plant': [], 'bed': [], 'dining table': [], 'toilet': [], 'tv': [], 'laptop': [], 'mouse': [], 'remote': [], 
               'keyboard': [], 'cell phone': [], 'microwave': [], 'oven': [], 'toaster': [], 'sink': [], 'refrigerator': [], 'book': [], 
               'clock': [], 'vase': [], 'scissors': [], 'teddy bear': [], 'hair drier': [], 'toothbrush': [] }
COCO_CLASS_NAMES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
            'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
            'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
            'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
            'hair drier', 'toothbrush']

COCO_ROOT_TRAIN = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/train/"
COCO_ROOT_TEST = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/test/"
COCO_ROOT_VAL = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/val/"
COCO_RAW_TRAIN = "C:/Users/phiho/Projects/Dataset/MS-COCO/train2017"
COCO_RAW_TEST = "C:/Users/phiho/Projects/Dataset/MS-COCO/test2017"

import os
for i in range(0, len(COCO_CLASS_NAMES)):
    imgs_path_train = COCO_ROOT_TRAIN + COCO_CLASS_NAMES[i]
    for root, dirs, files in os.walk(imgs_path_train):
        for file in files:
            COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]].append(os.path.join(root, file).split("//")[-1])

# Utils
def get_ground_truth(img_path):
    if '//' in img_path:
        img_path = img_path.split("//")[-1]
    if '/' in img_path:
        img_path = img_path.split("/")[-1]
    if '\\' in img_path:
        img_path = img_path.split("\\")[-1]

    for i in range(0, len(COCO_CLASS_NAMES)):
        result_string = ''.join(COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]])
        if img_path in result_string:
            return i
    return None    

In [2]:
import torch
import torchvision
import cv2
import numpy as np
import torch.nn as nn

class IdentityLinear(nn.Module):
    def __init__(self, input_size=80):
        super(IdentityLinear, self).__init__()
        self.linear = nn.Linear(input_size, input_size, bias=True)  # Input size to output size

        # Initialize weight matrix as identity and bias vector as zeros
        self.linear.weight.data = torch.eye(input_size, dtype=torch.float64)
        self.linear.bias.data = torch.zeros(input_size, dtype=torch.float64)

    def forward(self, x):
        return self.linear(x)

ROOT = './yolov8'  # YOLOv8 root directory
convert_gradient = IdentityLinear()

from yolov8.models.common import DetectMultiBackend
from yolov8.utilss.general import (
    non_max_suppression,
)
from yolov8.utilss.torch_utils import smart_inference_mode

@smart_inference_mode()
def target_model(dataset=None, conf_thres=0, device=None, one_img=False, label_mode=False):
    if device == None: device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
    weights = ROOT + '/' + "yolov8s.pt"
    data = ROOT + '/' + "data/coco128.yaml"
    imgsz=(320, 320)
    resize = torchvision.transforms.Resize(imgsz)

    # Load model
    model = DetectMultiBackend(weights, device=device)
    stride, names, pt = model.stride, model.names, model.pt

    if one_img:
        im = dataset
        # Run inference
        batch = 1
        model.warmup(imgsz=(1 if pt or model.triton else batch, 3, *imgsz))  # warmup

        im = resize(im)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im = im[None]  # expand for batch dimim = im.permute(2, 0, 1) / 255
        

        pred = model(im)
        pred = non_max_suppression(pred, conf_thres=conf_thres)
           
        c_ = 0
        res = np.zeros((batch, 80))
        for i, det in enumerate(pred):
            if len(det):
                for *xyxy, conf, cls in reversed(det):
                    c = int(cls)
                    confidence = float(conf)
                    if res[c_][c] < confidence:
                        res[c_][c] = confidence

        # print(names[np.argmax(res[c_])])
        return np.argmax(res[c_])


    else:
        if isinstance(dataset, str):        
            # Run inference
            batch = 1
            model.warmup(imgsz=(1 if pt or model.triton else batch, 3, *imgsz))  # warmup

            im = cv2.imread(dataset)
            im = torch.from_numpy(im).to(model.device)
            im = im.permute(2, 0, 1) / 255
            im = resize(im)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im = im[None]  # expand for batch dim
            pred = model(im)
            pred = non_max_suppression(pred, conf_thres=conf_thres)
            c_ = 0
            res = np.zeros((batch, 80))
            for i, det in enumerate(pred):
                if len(det):
                    for *xyxy, conf, cls in reversed(det):
                        c = int(cls)
                        confidence = float(conf)
                        if res[c_][c] < confidence:
                            res[c_][c] = confidence

            if label_mode: return names[np.argmax(res[c_])]
            return np.argmax(res[c_])

        else:
            # Run inference
            batch = dataset.shape[0]
            model.warmup(imgsz=(1 if pt or model.triton else batch, 3, *imgsz))  # warmup

            c_ = 0
            res = np.zeros((batch, 80))
            
            for im in dataset:
                im = resize(im)
                im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
                im = im[None]  # expand for batch dim

                # Inference
                pred = model(im)
                
                # NMS
                pred = non_max_suppression(pred, conf_thres=conf_thres)

                # Process predictions
                for i, det in enumerate(pred):
                    if len(det):
                        # Write results
                        for *xyxy, conf, cls in reversed(det):
                            c = int(cls)
                            confidence = float(conf)
                            if res[c_][c] < confidence:
                                res[c_][c] = confidence

                            # print(c_, label, confidence_str)
                c_ = c_ + 1

            return torch.from_numpy(res).to(device)

In [18]:
KCOUNT = 7

import shutil 
import os 

# Xoá preview
preview_path = "C:/Users/phiho/Projects/GAN_Yolov8/preview"
if os.path.exists(preview_path):
    shutil.rmtree(preview_path)

# Tạo folder preview mới
os.makedirs(preview_path)
for i in range(1, KCOUNT+1):
    sub_folder_name = f'0_{i}'
    sub_folder_path = os.path.join(preview_path, sub_folder_name)
    os.makedirs(sub_folder_path)

from skimage.metrics import structural_similarity as compare_ssim
import torch
import torchvision.datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from Gan_model import Generator
import cv2
import numpy as np
import time
import torch.nn.functional as F
from torchvision.transforms.functional import to_tensor
from skimage import io
import math
import os

start_time = time.time()
print(start_time,'running')
use_cuda=True
image_nc=3
batch_size = 1

gen_input_nc = image_nc

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda")

# load the pretrained model
print('=================== yolov8s.pt model ===============================================')
# load the generator of adversarial examples
pretrained_generator_path = 'C:/Users/phiho/Projects/GAN_Yolov8/models/netG.57'
print(pretrained_generator_path)
pretrained_G = Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()
for kcount in range(1,KCOUNT+1):
    start_time_gen = time.time()
    print(start_time_gen,' running gen ',kcount)

    # test adversarial examples in  testing dataset
    coco_dataset = torchvision.datasets.ImageFolder('./dataset/test_dog_cat_person', transform=transforms.ToTensor())
    test_dataloader = DataLoader(coco_dataset, batch_size=1, shuffle=False, drop_last=True)

    print("len(test_dataloader): ", len(test_dataloader))
    num_correct = 0
    num_all = 0
    spsnr = 0
    sssim = 0
    count = 0
    sl0 = 0.0
    sl1 = 0.0
    sl2 = 0.0

    CLASS_NAMES_DICT = { 'person': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bicycle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'car': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'motorcycle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'airplane': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bus': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'train': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'truck': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'boat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'traffic light': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'fire hydrant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'stop sign': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'parking meter': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bench': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bird': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'dog': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'horse': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sheep': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cow': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'elephant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bear': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'zebra': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'giraffe': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'backpack': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'umbrella': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'handbag': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tie': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'suitcase': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'frisbee': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'skis': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'snowboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sports ball': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'kite': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'baseball bat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'baseball glove': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'skateboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'surfboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tennis racket': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bottle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'wine glass': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cup': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'fork': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'knife': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'spoon': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bowl': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'banana': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'apple': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sandwich': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'orange': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'broccoli': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'carrot': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'hot dog': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'pizza': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'donut': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cake': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'chair': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'couch': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'potted plant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bed': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'dining table': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toilet': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tv': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'laptop': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'mouse': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'remote': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'keyboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cell phone': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'microwave': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'oven': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toaster': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sink': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'refrigerator': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'book': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'clock': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'vase': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'scissors': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'teddy bear': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'hair drier': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toothbrush': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0} }

    print('0.',kcount,'*perturbation')
    for i, data in enumerate(test_dataloader, 0):
        test_img, test_label = data
        name, _ = test_dataloader.dataset.samples[i]
        
        # test_label
        name = name.split("\\")[-1]
        test_label = torch.tensor(np.array([get_ground_truth(name)]))
        
        if COCO_CLASS_NAMES[test_label[0].cpu()] not in ['dog', 'cat', 'person']:
            continue

        test_img, test_label = test_img.to(device), test_label.to(device)

        targeted_rslt = target_model(test_img, device=device)
        targeted_lb = torch.argmax(targeted_rslt, 1)
        
        if (torch.sum(test_label==targeted_lb, 0) == 0):
            continue
        targeted_score = torch.nn.Softmax()(targeted_rslt[0])[targeted_lb[0]].detach().cpu().numpy()

        perturbation = pretrained_G(test_img)
        perturbation = torch.clamp(perturbation, -0.3, 0.3)
        # thay doi mean thhanh mediant >>
        perturbation_mean = perturbation.mean(dim=[1,2,3])
        perturbation[0] = perturbation[0] - perturbation_mean[0]

        #adv_img = test_img + k(tu 0.1 den 1.0) *perturbation
        adv_img = test_img + int(kcount)*0.1*perturbation

        adv_img = torch.clamp(adv_img, 0, 1)

        pred_rslt = target_model(adv_img)
        pred_lab = torch.argmax(pred_rslt, 1)
        pred_scores = torch.nn.Softmax()(pred_rslt[0])[pred_lab[0]].detach().cpu().numpy()

        num_all += len(test_label)
        current_correct = torch.sum(pred_lab==test_label,0)
        num_correct += current_correct

        # quick test for batch size = 1
        CLASS_NAMES_DICT[COCO_CLASS_NAMES[test_label[0].detach().cpu().numpy()]]["gt"] += 1
        if (current_correct.item() != 0):
            continue
        CLASS_NAMES_DICT[COCO_CLASS_NAMES[test_label[0].detach().cpu().numpy()]]["adv_succeed"] += 1
        if (current_correct.item() != 0):
            continue

        count += 1

        cv2.imwrite(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-0origin -label_{test_label[0].detach().cpu().numpy()}-classname_{COCO_CLASS_NAMES[test_label[0].detach().cpu().numpy()]}.jpg",
                            cv2.cvtColor(test_img[0].permute((1,2,0)).detach().cpu().numpy()*255, cv2.COLOR_RGB2BGR))

        cv2.imwrite(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-1targerted-label_{targeted_lb[0].detach().cpu().numpy()}_{targeted_score}-classname_{COCO_CLASS_NAMES[targeted_lb[0].detach().cpu().numpy()]}.jpg",
                            cv2.cvtColor(test_img[0].permute((1,2,0)).detach().cpu().numpy()*255, cv2.COLOR_RGB2BGR))

        cv2.imwrite(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-2pert.jpg",
                            cv2.cvtColor(perturbation[0].permute((1,2,0)).detach().cpu().numpy()*255, cv2.COLOR_RGB2BGR))

        cv2.imwrite(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-3adv-label_{pred_lab[0].detach().cpu().numpy()}_{pred_scores}-classname_{COCO_CLASS_NAMES[pred_lab[0].detach().cpu().numpy()]}.jpg",
                            cv2.cvtColor(adv_img[0].permute((1,2,0)).detach().cpu().numpy()*255, cv2.COLOR_RGB2BGR))
        image1 = cv2.imread(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-0origin -label_{test_label[0].detach().cpu().numpy()}-classname_{COCO_CLASS_NAMES[test_label[0].detach().cpu().numpy()]}.jpg", cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(f"C:/Users/phiho/Projects/GAN_Yolov8/preview/0_{kcount}/{name}-3adv-label_{pred_lab[0].detach().cpu().numpy()}_{pred_scores}-classname_{COCO_CLASS_NAMES[pred_lab[0].detach().cpu().numpy()]}.jpg", cv2.IMREAD_GRAYSCALE)
        
        ssim_score = compare_ssim(image1, image2)
        print(f"SSIM score: {ssim_score}","   id  ",count)
        
        # Chuyển hình ảnh thành tensor PyTorch
        image1_tensor = to_tensor(image1).unsqueeze(0)
        image2_tensor = to_tensor(image2).unsqueeze(0)
 
        # tinh L0 L1 L2 LPnorm
        l1 = float(F.l1_loss(image1_tensor,image2_tensor))
        l0 = float(torch.norm(abs(image1_tensor - image2_tensor)))    
        # Tính l2
        l2 = float(F.mse_loss(image1_tensor, image2_tensor).cpu().numpy())    
        
        # Chuyển sang kiểu dữ liệu float32 và chuẩn hóa giá trị về khoảng [0, 1]
        image1_tensor = image1_tensor.type(torch.float32) / 255.0
        image2_tensor = image2_tensor.type(torch.float32) / 255.0
        mse = F.mse_loss(image1_tensor, image2_tensor)
        psnr = 10 * torch.log10(1.0 / torch.sqrt(F.mse_loss(image1_tensor, image2_tensor)))
        print(f"PSNR: {psnr.item()}","   id  ",count)
        if(math.isinf(psnr.item())==False):
            spsnr += psnr.item()
        print("Sum SPSNR: ", spsnr)
        
        # tinh L0 L1 L2 LPnorm
        # Tính l2
        l2 = float(mse.cpu().numpy())

        print(f"L0 score: {l0:.21f}","   id  ",count)
        print(f"L1 score: {l1:.21f}","   id  ",count)
        print(f"L2 score: {l2:.21f}","   id  ",count)

        sl0 += l0
        sl1 += l1
        sl2 += l2
    
        print("SUM ", count," L0  : ", f"{sl0:.21f}"," -L1 : ", f"{sl1:.21f}"," -L2 : ", f"{sl2:.21f}" )
        print("Sum SPSNR: ", spsnr)
        sssim += ssim_score
        print("Sum SSSIM: ", sssim)
        
    print('SSIM: TB ',sssim/count) 
    print('PSNR: TB ',spsnr/count) 
    print("AVG L0  : ", f"{sl0/count:.21f}"," -L1 : ", f"{sl1/count:.21f}"," -L2 : ", f"{sl2/count:.21f}" )    
    print('Test dataset:')
    print('num_examples: ', num_all)
    print('num_correct: ', num_correct.item())
    print('adv_correct: ', count)
    print('accuracy of adv imgs in testing set: %f\n'%(num_correct.item()/num_all))
    print('adv succeeded %f\n'%(1-num_correct.item()/num_all))
    print("person:", CLASS_NAMES_DICT['person'])
    print("dog:", CLASS_NAMES_DICT['dog'])
    print("cat:", CLASS_NAMES_DICT['cat'])
    for cls_name in COCO_CLASS_NAMES:
        if cls_name in ['dog', 'cat', 'person']:
            CLASS_NAMES_DICT[cls_name]["adv_succeed_%"] = CLASS_NAMES_DICT[cls_name]["adv_succeed"] / CLASS_NAMES_DICT[cls_name]["gt"]


    print(time.time()-start_time_gen)
    print('==================================================================')

print(time.time()-start_time)

1719629984.5563586 running
CUDA Available:  True
=================== yolov5s.pt model ===============================================
C:/Users/phiho/Projects/GAN_Yolov5/models/netG.57
1719629984.610893  running gen  1
len(test_dataloader):  1392
0. 1 *perturbation
SSIM score: 0.9268327958245817    id   1
PSNR: 43.996524810791016    id   1
Sum SPSNR:  43.996524810791016
L0 score: 0.011912780813872814178    id   1
L1 score: 0.000029494949558284134    id   1
L2 score: 0.000000001587432074679    id   1
SUM  1  L0  :  0.011912780813872814178  -L1 :  0.000029494949558284134  -L2 :  0.000000001587432074679
Sum SPSNR:  43.996524810791016
Sum SSSIM:  0.9268327958245817
SSIM score: 0.9793099839842121    id   2
PSNR: 43.23851776123047    id   2
Sum SPSNR:  87.23504257202148
L0 score: 0.014184383675456047058    id   2
L1 score: 0.000036991208617109805    id   2
L2 score: 0.000000002250594821618    id   2
SUM  2  L0  :  0.026097164489328861237  -L1 :  0.000066486158175393939  -L2 :  0.0000000038380

In [19]:
KCOUNT = 7

import shutil 
import os
import json
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.metrics import structural_similarity as compare_ssim
import torch
import torch.nn.functional as F
from torchvision.transforms.functional import to_tensor
import math


# Xoá preview
preview_path = "C:/Users/phiho/Projects/GAN_Yolov5/preview_big_image"
if os.path.exists(preview_path):
    shutil.rmtree(preview_path)

# Tạo folder preview mới
os.makedirs(preview_path)
for i in range(1, KCOUNT+1):
    sub_folder_name = f'0_{i}'
    sub_folder_path = os.path.join(preview_path, sub_folder_name)
    os.makedirs(sub_folder_path)

def load_json_from_txt(path):
    with open(path, 'r') as file:
        data = file.read()
    return json.loads(data)

def get_set_crop_img(image_names):
    unique_prefixes = set()
    for image_name in image_names:
        prefix = image_name.split('_')[0]
        unique_prefixes.add(prefix)
    return unique_prefixes

def get_img_in4_in_txt(img_name, txt_path):
    with open(txt_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.split()
            if img_name == parts[0]:
                return { 'label' : int(parts[1]), 'x_center' : float(parts[2]), 'y_center' : float(parts[3]), 
                        'w_bbox' : float(parts[4]), 'h_bbox' : float(parts[5]), 'w_img' : int(parts[6]), 'h_img' : int(parts[7]) }
    return None

log_crop_path = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/log_crop_coco81.txt"
def get_img_all_set(path):
    res = []
    imgs = os.listdir(path)
    all_imgs = get_set_crop_img([x for x in imgs if "0origin" in x])
    for img in all_imgs:
        perts = [os.path.join(path, x) for x in imgs if "2pert" in x and img in x]
        advs = [os.path.join(path, x) for x in imgs if "3adv" in x and img in x]
        origins = [{get_path_from_name(x.split("-3adv")[0]) : get_img_in4_in_txt(x.split("-3adv")[0], log_crop_path)} 
                   for x in imgs if "3adv" in x and img in x]
        res.append({img : [origins, perts, advs]})
    return res

def get_img_in4_in_dict(data):
    for key, value in data.items():
        return key, value

def get_path_from_name(name):
    if '_' in name:
        for subfolder in os.listdir(COCO_ROOT_TRAIN):
            for img in os.listdir(os.path.join(COCO_ROOT_TRAIN, subfolder)):
                if (name == img) or (name + ".jpg" == img):
                    return os.path.join(os.path.join(COCO_ROOT_TRAIN, subfolder), img)
            for img in os.listdir(os.path.join(COCO_ROOT_TEST, subfolder)):
                if (name == img) or (name + ".jpg" == img):
                    return os.path.join(os.path.join(COCO_ROOT_TEST, subfolder), img)

    for img in os.listdir(COCO_RAW_TRAIN):
        if (name == img) or (name + ".jpg" == img):
            return os.path.join(COCO_RAW_TRAIN, img)
    for img in os.listdir(COCO_RAW_TEST):
        if (name == img) or (name + ".jpg" == img):
            return os.path.join(COCO_RAW_TEST, img)
    return None

def show_img_from_data(data):
    image_rgb = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

#-------------------------------------------------------------------------------------------------------------#
#                     Ghép ngược ảnh nhỏ sang ảnh lớn từ các folder 0_1, 0_2, ..., 0_k
#-------------------------------------------------------------------------------------------------------------#

for k in range(1, KCOUNT+1):
    spsnr = 0
    sssim = 0
    count = 0
    sl0 = 0.0
    sl1 = 0.0
    sl2 = 0.0

    CLASS_NAMES_DICT = { 'person': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bicycle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'car': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'motorcycle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'airplane': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bus': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'train': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'truck': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'boat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'traffic light': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'fire hydrant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'stop sign': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'parking meter': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bench': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bird': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'dog': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'horse': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sheep': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cow': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'elephant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bear': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'zebra': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'giraffe': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'backpack': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'umbrella': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'handbag': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tie': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'suitcase': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'frisbee': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'skis': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'snowboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sports ball': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'kite': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'baseball bat': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'baseball glove': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'skateboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'surfboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tennis racket': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bottle': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'wine glass': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cup': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'fork': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'knife': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'spoon': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bowl': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'banana': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'apple': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sandwich': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'orange': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'broccoli': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'carrot': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'hot dog': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'pizza': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'donut': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cake': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'chair': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'couch': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'potted plant': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'bed': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'dining table': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toilet': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'tv': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'laptop': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'mouse': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'remote': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'keyboard': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'cell phone': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'microwave': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'oven': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toaster': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'sink': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'refrigerator': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'book': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'clock': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'vase': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'scissors': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'teddy bear': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'hair drier': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0}, 'toothbrush': {"gt": 0, "adv_succeed": 0, "adv_succeed_%": 0} }

    print('0.',k,'*perturbation')

    src_folder_path = f"C:/Users/phiho/Projects/GAN_Yolov5/preview/0_{k}"
    des_folder_path = f"C:/Users/phiho/Projects/GAN_Yolov5/preview_big_image/0_{k}"
    data = get_img_all_set(src_folder_path)

    for id in range(len(data)):
        img_big, img_big_data = get_img_in4_in_dict(data[id])
        
        if get_path_from_name(img_big) == None: continue
        else: count += 1
        
        img_big_org = cv2.imread(get_path_from_name(img_big))
        img_big_pert = np.zeros(img_big_org.shape, dtype=np.uint8)
        img_big_adv = cv2.imread(get_path_from_name(img_big))
        oris = img_big_data[0]
        perts = img_big_data[1]
        advs = img_big_data[2]

        for i in range(len(oris)):
            img_small, img_small_data = get_img_in4_in_dict(oris[i])
            img_small_org = cv2.imread(img_small)
            img_small_pert = cv2.imread(perts[i])
            img_small_adv = cv2.imread(advs[i])
            
            x_center = img_small_data['x_center']
            y_center = img_small_data['y_center']
            w_bbox = img_small_data['w_bbox']
            h_bbox = img_small_data['h_bbox']
            w_img = img_small_data['w_img']
            h_img = img_small_data['h_img']
            
            x_center_new = x_center * w_img
            y_center_new = y_center * h_img
            w_bbox_new = w_bbox * w_img
            h_bbox_new = h_bbox * h_img
            x1 = x_center_new - w_bbox_new/2
            y1 = y_center_new + h_bbox_new/2
            x2 = x_center_new + w_bbox_new/2
            y2 = y_center_new - h_bbox_new/2
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            img_small_pert = cv2.resize(img_small_pert, (x2-x1, y1-y2))
            img_small_adv = cv2.resize(img_small_adv, (x2-x1, y1-y2))

            img_big_pert[y2:y1,x1:x2] = img_small_pert
            img_big_adv[y2:y1,x1:x2] = img_small_adv

        cv2.imwrite(os.path.join(des_folder_path, f"{img_big}-org.jpg"), img_big_org)
        cv2.imwrite(os.path.join(des_folder_path, f"{img_big}-pert.jpg"), img_big_pert)
        cv2.imwrite(os.path.join(des_folder_path, f"{img_big}-adv.jpg"), img_big_adv)

        img_big_org = cv2.cvtColor(img_big_org, cv2.COLOR_BGR2GRAY)
        img_big_adv = cv2.cvtColor(img_big_adv, cv2.COLOR_BGR2GRAY)

        ssim_score = compare_ssim(img_big_org, img_big_adv)
        print(f"SSIM score: {ssim_score}","   id  ",count)
        
        # Chuyển hình ảnh thành tensor PyTorch
        image1_tensor = to_tensor(img_big_org).unsqueeze(0)
        image2_tensor = to_tensor(img_big_adv).unsqueeze(0)
        
        # Chuyển sang kiểu dữ liệu float32 và chuẩn hóa giá trị về khoảng [0, 1]
        image1_tensor = image1_tensor.type(torch.float32) / 255.0
        image2_tensor = image2_tensor.type(torch.float32) / 255.0

        l1 = float(F.l1_loss(image1_tensor,image2_tensor))
        l0 = float(torch.norm(abs(image1_tensor - image2_tensor)))
        
        mse = F.mse_loss(image1_tensor, image2_tensor)
        psnr = 10 * torch.log10(1.0 / torch.sqrt(mse))
        print(f"PSNR: {psnr.item()}","   id  ",count)
        if(math.isinf(psnr.item())==False):
            spsnr += psnr.item()
        print("Sum SPSNR: ", spsnr)
        
        # tinh L0 L1 L2 LPnorm
        # Tính l2
        l2 = float(mse.cpu().numpy())

        print(f"L0 score: {l0:.21f}","   id  ",count)
        print(f"L1 score: {l1:.21f}","   id  ",count)
        print(f"L2 score: {l2:.21f}","   id  ",count)

        sl0 += l0
        sl1 += l1
        sl2 += l2
    
        print("SUM ", count," L0  : ", f"{sl0:.21f}"," -L1 : ", f"{sl1:.21f}"," -L2 : ", f"{sl2:.21f}" )
        print("Sum SPSNR: ", spsnr)
        sssim += ssim_score
        print("Sum SSSIM: ", sssim)
        
    print('SSIM: TB ',sssim/count) 
    print('PSNR: TB ',spsnr/count) 
    print("AVG L0  : ", f"{sl0/count:.21f}"," -L1 : ", f"{sl1/count:.21f}"," -L2 : ", f"{sl2/count:.21f}" )    
    print('==================================================================')

0. 1 *perturbation
SSIM score: 0.9928461823537521    id   1
PSNR: 43.757110595703125    id   1
Sum SPSNR:  43.757110595703125
L0 score: 0.021982690319418907166    id   1
L1 score: 0.000009011238944367506    id   1
L2 score: 0.000000001772465507699    id   1
SUM  1  L0  :  0.021982690319418907166  -L1 :  0.000009011238944367506  -L2 :  0.000000001772465507699
Sum SPSNR:  43.757110595703125
Sum SSSIM:  0.9928461823537521
SSIM score: 0.999624562132035    id   2
PSNR: 49.85871887207031    id   2
Sum SPSNR:  93.61582946777344
L0 score: 0.005725834053009748459    id   2
L1 score: 0.000001260632757293934    id   2
L2 score: 0.000000000106722630733    id   2
SUM  2  L0  :  0.027708524372428655624  -L1 :  0.000010271871701661439  -L2 :  0.000000001879188138432
Sum SPSNR:  93.61582946777344
Sum SSSIM:  1.9924707444857872
SSIM score: 0.9747179072162828    id   3
PSNR: 42.63531494140625    id   3
Sum SPSNR:  136.2511444091797
L0 score: 0.030212314799427986145    id   3
L1 score: 0.0000176867561094